# Aprendizado Profundo - UFMG

## Preâmbulo

O código abaixo consiste dos imports comuns. Além do mais, configuramos as imagens para ficar de um tamanho aceitável e criamos algumas funções auxiliares. No geral, você pode ignorar a próxima célula.

In [1]:
# -*- coding: utf8

import matplotlib.pyplot as plt

import torch

import numpy as np

plt.rcParams['figure.figsize']  = (18, 10)
plt.rcParams['axes.labelsize']  = 20
plt.rcParams['axes.titlesize']  = 20
plt.rcParams['legend.fontsize'] = 20
plt.rcParams['xtick.labelsize'] = 20
plt.rcParams['ytick.labelsize'] = 20
plt.rcParams['lines.linewidth'] = 4

In [2]:
plt.ion()

plt.style.use('seaborn-colorblind')
plt.rcParams['figure.figsize']  = (12, 8)

O código de GD abaixo está pré-preparado para a primeira parte da tarefa. Não precisa mudar o mesmo!

In [3]:
def gd(d_fun, loss_fun, X, y, lambda_=0.01, tol=0.00001, max_iter=10000):
    theta = torch.ones(Y.shape[1],X.shape[1])
    theta.requires_grad_(True)
    #print('Iter {}; theta = '.format(0), theta)
    
    old_lf = np.inf
    i = 0
    while True:
        # Computar as derivadas
        theta.requires_grad_(True)    
        grad = d_fun(X,theta,y)

        # Atualizar
        with torch.no_grad():
            theta_novo = theta - lambda_ * grad
        
        #Parar quando o erro convergir
        lf = loss_fun(X, theta, y)
        if torch.abs(old_lf - lf) <= tol:
            break
        
         #Atualizar parâmetros e erro
        theta = theta_novo
        old_lf = lf
        
        # Informação de debug
        #print('Iter {}; theta = '.format(i+1), theta)
        i += 1
        if i == max_iter:
            break
        
    return theta

Para testar o resultado dos seus algoritmos vamos usar o módulo testing do numpy.

In [4]:
from numpy.testing import assert_equal
from numpy.testing import assert_almost_equal
from numpy.testing import assert_array_almost_equal

## Aula 03 - Softmax para Imagens

Continuando da aula anterior, vamos implementar uma regressão logística (softmax) para várias classes. Além do mais, vamos fazer uso da mesma para algumas tarefas classificação de imagens.

Para iniciar a transição para o mundo de Deep Learning, vamos implementar uma nova logística (não será mais do zero) usando as camadas que o pytorch já traz prontas.

!!? (Qual sera o equivalente em pytorch) Antes de iniciar o notebook, sugiro uma revisão do [Capítulo 3](d2l.ai).

## Softmax em PyTorch/PyTorch NN

A função softmax pode ser utilizada para problemas multiclasse. No exemplo abaixo, temos a função softmax exemplificada. Diferente dos casos anteriores, aqui nós temos uma matriz de parâmetros: $\mathbf{\Theta}$. Cada coluna da matriz $\mathbf{\Theta}_y$ contém os parâmetros para uma classe $y$.

$$p(y|\mathbf{x}_i, \mathbf{\theta}_y) = \mathrm{softmax}(\mathbf{x}^t_i) = \frac{\exp(\mathbf{x}^t_i \mathbf{\theta}_y)}{\sum_{y'} \exp(\mathbf{x}^t_{i} \mathbf{\theta}_{y'})}$$

Vamos pensar no caso que temos 10 classes e 20 atributos:
  * O tamanho e $\mathbf{X}$ é `(n, 20)`, `n` é o número de exemplos
  * O tamanho e $\mathbf{\Theta}$ é `(c, 20)`, onde `c` é o número de classes
  * $\mathbf{\Theta}^t$ é `(20, c)`
  * $\mathbf{X} \mathbf{\Theta}^t$ é `(n, c)`. Uma probabilidade para cada classe. Tal produto interno é representado por uma linha de $\mathbf{X}$, $\mathbf{x}^t_i$ multiplicado pela coluna de $\mathbf{\Theta}$, $\mathbf{\theta}_y$.
  
O softmax é basicamente uma matriz onde aplicamos uma exponencial em toda célula: $e^{\mathbf{X} \mathbf{\Theta}^t}$. Depois normalizamos por classe. Assim voltamos para probabilidades.

Primeiro, sem usar o modulo nn do pytorch ainda, implemente:
   1. Uma função softmax
   1. Uma função de perda
   1. Derivadas em pytorch (autograd)

Antes disso, vamos usar nossos blobs de sempre. Essa deve ser a última aula com os mesmo :-(

No nosso novo exemplo temos 200 amostras, 20 features e 10 classes!

In [5]:
from sklearn import datasets
state = np.random.seed(20190187)

X, y = datasets.make_blobs(n_samples=200, n_features=20, centers=2)
X = torch.tensor(X)
y = torch.tensor(y)

In [6]:
X.shape

torch.Size([200, 20])

In [7]:
len(set(y.numpy()))

2

Implemente a função softmax.

In [8]:
def softmax(X, Theta):
    '''
    Aqui Theta é a matriz de parâmetros e X uma matriz.
    Seu código deve retornar um vetor de previsões para toda linha de X.
    '''
    P = torch.exp(torch.matmul(X.double(), (Theta.T).double()))
    return (P.T / P.sum(axis=1)).T.double()

In [9]:
# testes, não apague!

# Se X tem tamanho (n, f) a matriz theta é (c, f). c é o numéro de clases.
# X.T é (f, c). Assim X @ Theta.T -> (n, c)
# O resultado do softmax é a probabilidade de cada classe

Theta = torch.randn(4, X.shape[1])
P = softmax(X, Theta)
assert_equal((200, 4), P.shape)
assert_almost_equal(np.ones(len(P)), P.sum(axis=1).numpy(), 4) # verifica se toda linha soma == 1.

In [10]:
Theta = torch.randn(4, X.shape[1])
P = softmax(X, Theta)
P

tensor([[2.4867e-08, 1.5216e-01, 9.4605e-11, 8.4784e-01],
        [9.9834e-01, 1.6629e-03, 2.5770e-18, 4.8671e-12],
        [1.5081e-14, 4.9174e-04, 1.3891e-14, 9.9951e-01],
        [3.2386e-04, 9.9968e-01, 1.2864e-15, 9.7512e-14],
        [1.0000e+00, 4.7578e-06, 3.5355e-15, 6.3521e-15],
        [6.4200e-12, 7.1688e-05, 2.4624e-13, 9.9993e-01],
        [9.9810e-01, 1.8981e-03, 1.4810e-11, 3.4749e-10],
        [9.9983e-01, 1.7398e-04, 7.9975e-13, 5.0942e-16],
        [1.3654e-10, 3.3068e-04, 5.0270e-07, 9.9967e-01],
        [3.6507e-01, 6.3493e-01, 4.3364e-14, 3.2655e-11],
        [2.8886e-09, 2.9250e-01, 1.2185e-08, 7.0750e-01],
        [1.7854e-03, 9.9821e-01, 1.1592e-17, 1.5827e-15],
        [9.9998e-01, 2.4518e-05, 6.9354e-13, 1.1573e-10],
        [1.9003e-15, 2.5687e-05, 1.8866e-12, 9.9997e-01],
        [1.4068e-01, 8.5932e-01, 2.4998e-12, 2.6254e-11],
        [3.3574e-05, 9.9997e-01, 2.9717e-19, 3.1242e-11],
        [6.3119e-13, 2.8449e-05, 1.8970e-12, 9.9997e-01],
        [9.696

Agora implemente uma função de perda de entropia cruzada. Um truque para implementar a mesma é pensar nas respostas como uma matrix $\mathbf{Y}$. O tamanho de $\mathbf{Y}$ é `(n, c)`, ou seja, exemplos por classes. Cada vetor das linhas da matriz, simplesmente $\mathbf{y}_i$, é da forma one-hot `0, 0, 0, 1, 0`. Neste formato, apenas a classe do exemplo está setada como 1, todo o resto é zero. No exemplo, o elemento $i$ é da classe `3` (assumindo que a primeira classe é `0`). Vamos converter nossa resposta y em tal matriz.

In [11]:
import sklearn.preprocessing
hot = sklearn.preprocessing.OneHotEncoder(sparse=False)
# y[:, None] adiciona uma dimensão, vira uma matriz (n, 1). Cada linha é uma classe, e.g.: [[1], [0], [0], ...]
# O fit_transform vai converter em uma matriz (n, c).
Y = hot.fit_transform(y[:, None]) 
# fazendo Y ser mxnet
Y = torch.tensor(Y)
Y

/home/joaocouto/anaconda3/envs/temp/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


tensor([[1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1

In [12]:
print(Y.shape)

torch.Size([200, 2])


In [13]:
# fazendo X ser um tensor pytorch
X = torch.tensor(X)

/home/joaocouto/anaconda3/envs/temp/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [14]:
print(X.shape)

torch.Size([200, 20])


Agora podemos definir a entropia cruzada do caso multiclasse. Sendo:

$$\hat{\mathbf{p}}_i = < p(y_0|\mathbf{x}_i, \mathbf{\theta}_y)), \quad  p(y_1|\mathbf{x}_i, \mathbf{\theta}_y)), \quad \cdots, \quad p(y_{n-1}|\mathbf{x}_i, \mathbf{\theta}_y))>^t$$

Um vetor coluna com a probabilidade de cada classe para o exemplo $i$. Ou seja, o vetor tem `c` linhas. $\mathbf{y}^t_i$ é um vetor linha one-hot: e.g., `0, 0, 0, 1, 0`. O produto interno dos dois retorna a probabilidade da classe correta para cada exemplo: $\mathbf{y}^t_i \hat{\mathbf{p}}_i$. A média do log de cada probabilidade é a entropia cruzada!

$$CE(\theta \mid y_i, \mathbf{x}_i) = n^{-1} \sum_i \mathbf{y}^t_i \log(\, \hat{\mathbf{p}}_i\, )$$

Usando sua matriz `Y` acima. Basta fazer `Y * P` (multiplicação por elemento) para zerar qualquer probabilidade da classe errada. Depois disso, um `.sum(axis=1)` faz o produto interno para todos os exemplos.

Implemente a função `cross_entropy`.

In [15]:
def loss(X, Theta, Y):
    P = softmax(X, Theta)
    p = (Y * P).sum(axis=1)
    return -torch.log(p).mean()

In [16]:
# Testes
# O valor tem que ser um único número positivo
for _ in range(100):
    Theta = torch.randn(2, X.shape[1])
    assert(loss(X, Theta, Y) >= 0)
Theta.requires_grad_(True)


tensor([[ 0.2756, -0.2814,  0.7578, -2.6468, -0.8414, -1.5610, -0.9591, -1.0010,
         -0.4572, -1.1511,  0.5597, -1.1143,  0.3191, -0.6513, -0.2658,  0.3625,
         -1.5865, -1.6491,  0.6605,  0.6632],
        [-0.6167, -1.5550,  0.4883, -1.1062, -0.2460,  0.1895, -0.0418,  0.5089,
         -1.0788,  0.0986,  0.1188, -0.9229,  1.6993, -0.0484,  0.2143, -0.4027,
         -1.1465, -0.5094,  0.9355,  0.9562]], requires_grad=True)

In [17]:
def derivadas(X, Theta, Y):
    l = loss(X, Theta, Y)
    l.backward()
    return Theta.grad

In [18]:
loss(X, Theta, Y)

tensor(0.0738, dtype=torch.float64, grad_fn=<NegBackward>)

In [19]:
derivadas(X, Theta, Y)

tensor([[-0.1366,  0.0673,  0.0330, -0.2354,  0.1519,  0.1490, -0.0028, -0.0539,
         -0.0157, -0.2422,  0.0360, -0.2363,  0.0283, -0.1986,  0.1860, -0.1943,
         -0.0950,  0.2081, -0.0941,  0.1698],
        [ 0.1366, -0.0673, -0.0330,  0.2354, -0.1519, -0.1490,  0.0028,  0.0539,
          0.0157,  0.2422, -0.0360,  0.2363, -0.0283,  0.1986, -0.1860,  0.1943,
          0.0950, -0.2081,  0.0941, -0.1698]])

In [20]:
# Use essa função antes de executar o GD
def add_intercept(X):
    Xn = torch.zeros(X.shape[0], X.shape[1] + 1)
    Xn[:, 0]  = 1
    Xn[:, 1:] = X
    return Xn

In [21]:
Xn = add_intercept(X)
Theta = gd(derivadas, loss, Xn, Y, lambda_=0.005)

In [22]:
def previsoes(X, Theta):
    P = softmax(X, Theta)
    return P.argmax(axis=1)

In [23]:
y_p = previsoes(Xn, Theta)

In [24]:
y_p

tensor([0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0,
        1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1,
        0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0,
        1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0,
        1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1,
        1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1,
        0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1,
        1, 1, 0, 0, 0, 1, 0, 1])

In [25]:
from sklearn.metrics import classification_report
print(classification_report(y, y_p.numpy()))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       100
           1       1.00      1.00      1.00       100

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200



### Com PyTorch NN Module

In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F

A criação de uma rede com o PyTorch NN é trivial. Bastante similar com outras APIs de alto nível estilo Keras. Abaixo, defimos uma rede neural sequencial com uma única ativação densa de tamanho 10 saídas. Isto define uma função softmax de 10 classes.Você não precisa implementar nada. A ideia até mais em baixo é entender como a vida é mais simples ao usar a API do PyTorch NN

In [27]:
inputsDim, outDim = 28*28, 10 #Teremos imagens 28*28 e 10 classes de saida
model = nn.Sequential(
    nn.Linear(inputsDim, outDim, bias = True)
)

Agora indicamos que vamos fazer uso de uma entropia cruzada como função de perda.

In [28]:
criterion = nn.CrossEntropyLoss()

E que vamos treinar com gradiente descendente. SGD em particular!

In [29]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum = 0.5)

Todos os passos que fizemos na mão se reduzem a poucas linhas

#### Carregando Dados

Agora vamos carregar os dados do Fashion Mnist. Leia [este link](https://github.com/SurhanZahid/Fashion-MNIST-Classifier-Pytorch) para uma descrição da base. O PyTorch Vision consegue baixar ela da internet.

Antes de usar a base, vamos definir uma função de transformação para converter a mesma de imagens (28, 28) para vetores de (28 * 28) posições. Além do mais, converter as unidades para float32. Bibliotecas de aprendizado profundo como o pytorch são bem chatas com os tipos. É importante lembrar que os operadores de tensor recebam como input operandos de mesmo tipo

In [30]:
import torchvision
import torchvision.transforms as transforms

Carregando a Base.

In [31]:
#Transformando as imagens em tensores 2D 
trainset = torchvision.datasets.FashionMNIST(root = "./data", train = True, download = True, transform = transforms.ToTensor())
testset = torchvision.datasets.FashionMNIST(root = "./data", train = False, download = True, transform = transforms.ToTensor())

Com a classe `torch.utils.data.DataLoader` conseguimos iterar pela base em minibatches. Estamos usando aqui batches de tamanho 4

In [32]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=50, shuffle = True)
testloader = torch.utils.data.DataLoader(testset, batch_size=50, shuffle=False)

#### Treinando

Podemos treinar com um laço. Abaixo detalhamos o mesmo.

In [ ]:
max_epochs = 15
for epoch in range(max_epochs):
    #iterate through all the batches in each epoch
    for i, data in enumerate(trainloader, 0):
        #Coloca a rede em modo de treino   
        model.train()     
        inputs, labels = data  
        inputs = inputs.view(50,1,28*28)
        #Zera os gradientes (antes disso, eles contem o gradiente calculado no batch anterior)     
        optimizer.zero_grad()  
        
        #Forward pass     
        y_pred = model(inputs)
        y_pred = y_pred.view(len(labels),-1) #Efetiva um flatten das imagens de entrada 
        loss = criterion(y_pred.float(), labels)    
        
        #Backward pass     
        loss.backward()     
        optimizer.step()     
    print('epoch: ', epoch,' loss: ', loss.item())


epoch:  0  loss:  0.5188412666320801
epoch:  1  loss:  0.4720955789089203
epoch:  2  loss:  0.7403064966201782
epoch:  3  loss:  0.6215060949325562
epoch:  4  loss:  0.5209754705429077
epoch:  5  loss:  0.3291844129562378
epoch:  6  loss:  0.2333328276872635
epoch:  7  loss:  0.3119649291038513
epoch:  8  loss:  0.488322913646698
epoch:  9  loss:  0.6182224154472351
epoch:  10  loss:  0.468759685754776
epoch:  11  loss:  0.3305268883705139
epoch:  12  loss:  0.6013189554214478
epoch:  13  loss:  0.39560234546661377


#### Abaixo, avaliamos a acuracia em teste

In [ ]:
y_pred = []
y = []
for i,data in enumerate(testloader,0):
    inputs, labels = data
    inputs = inputs.view(50,1,28*28)
    y_pred.extend(model(inputs).argmax(2).tolist())
    y.extend(labels.tolist())



In [ ]:
y_pred = np.array([a[0] for a in y_pred])
y_pred

In [ ]:
y = np.array(y)

In [ ]:
(y_pred == y).mean()

## Perguntas

1. Altere o treino para computar a acurácia no mesmo
1. Brinque com a taxa de aprendizado do SGD e o número de iterações
1. Qual o impacto na acurácia do treino/teste?